# NFL Stadium vs. IHOP Proximity Analysis

This notebook calculates the distance between every NFL team's home stadium and the closest IHOP restaurant.

**Requirements:**
- Google Maps API Key (with Places API enabled)

In [12]:
# Install necessary libraries
!pip install googlemaps folium pandas

In [14]:
import googlemaps
import pandas as pd
import folium
import math
import os
import sys
from datetime import datetime
import getpass

# --- CONFIGURATION ---
API_KEY = None

# 1. Try loading from Environment Variable (Best for local)
if 'GOOGLE_MAPS_API_KEY' in os.environ:
    API_KEY = os.environ['GOOGLE_MAPS_API_KEY']
    print("🔑 Loaded key from Environment Variable")

# 2. Try loading from Google Colab Secrets
if not API_KEY:
    try:
        # Check if running in Colab
        if 'google.colab' in sys.modules:
            from google.colab import userdata
            API_KEY = userdata.get('GOOGLE_MAPS_API_KEY')
            print("🔑 Loaded key from Colab Secrets")
    except:
        # Catch ALL errors (including TimeoutException if it's a BaseException)
        pass

# 3. Fallback: Input prompt (secure for one-time use)
if not API_KEY:
    print("Enter your Google Maps API Key below (it will be hidden):")
    API_KEY = getpass.getpass()

if API_KEY:
    try:
        gmaps = googlemaps.Client(key=API_KEY)
        print("✅ Google Maps Client Initialized")
    except Exception as e:
        print(f"❌ Error initializing client: {e}")
else:
    print("⚠️ No API Key provided. The script will fail.")

Enter your Google Maps API Key below (it will be hidden):
✅ Google Maps Client Initialized


In [15]:
# --- DATA: NFL STADIUMS (2024 Season) ---
# Hardcoded coordinates for accuracy
nfl_stadiums = [
    {"Team": "Arizona Cardinals", "Stadium": "State Farm Stadium", "Lat": 33.5276, "Lng": -112.2626},
    {"Team": "Atlanta Falcons", "Stadium": "Mercedes-Benz Stadium", "Lat": 33.7554, "Lng": -84.4009},
    {"Team": "Baltimore Ravens", "Stadium": "M&T Bank Stadium", "Lat": 39.2780, "Lng": -76.6227},
    {"Team": "Buffalo Bills", "Stadium": "Highmark Stadium", "Lat": 42.7738, "Lng": -78.7870},
    {"Team": "Carolina Panthers", "Stadium": "Bank of America Stadium", "Lat": 35.2258, "Lng": -80.8528},
    {"Team": "Chicago Bears", "Stadium": "Soldier Field", "Lat": 41.8623, "Lng": -87.6167},
    {"Team": "Cincinnati Bengals", "Stadium": "Paycor Stadium", "Lat": 39.0955, "Lng": -84.5161},
    {"Team": "Cleveland Browns", "Stadium": "Cleveland Browns Stadium", "Lat": 41.5061, "Lng": -81.6995},
    {"Team": "Dallas Cowboys", "Stadium": "AT&T Stadium", "Lat": 32.7473, "Lng": -97.0945},
    {"Team": "Denver Broncos", "Stadium": "Empower Field at Mile High", "Lat": 39.7439, "Lng": -105.0201},
    {"Team": "Detroit Lions", "Stadium": "Ford Field", "Lat": 42.3400, "Lng": -83.0456},
    {"Team": "Green Bay Packers", "Stadium": "Lambeau Field", "Lat": 44.5013, "Lng": -88.0622},
    {"Team": "Houston Texans", "Stadium": "NRG Stadium", "Lat": 29.6847, "Lng": -95.4107},
    {"Team": "Indianapolis Colts", "Stadium": "Lucas Oil Stadium", "Lat": 39.7601, "Lng": -86.1639},
    {"Team": "Jacksonville Jaguars", "Stadium": "EverBank Stadium", "Lat": 30.3239, "Lng": -81.6373},
    {"Team": "Kansas City Chiefs", "Stadium": "GEHA Field at Arrowhead Stadium", "Lat": 39.0489, "Lng": -94.4839},
    {"Team": "Las Vegas Raiders", "Stadium": "Allegiant Stadium", "Lat": 36.0909, "Lng": -115.1833},
    {"Team": "Los Angeles Chargers", "Stadium": "SoFi Stadium", "Lat": 33.9534, "Lng": -118.3390},
    {"Team": "Los Angeles Rams", "Stadium": "SoFi Stadium", "Lat": 33.9534, "Lng": -118.3390},
    {"Team": "Miami Dolphins", "Stadium": "Hard Rock Stadium", "Lat": 25.9580, "Lng": -80.2389},
    {"Team": "Minnesota Vikings", "Stadium": "U.S. Bank Stadium", "Lat": 44.9735, "Lng": -93.2575},
    {"Team": "New England Patriots", "Stadium": "Gillette Stadium", "Lat": 42.0909, "Lng": -71.2643},
    {"Team": "New Orleans Saints", "Stadium": "Caesars Superdome", "Lat": 29.9511, "Lng": -90.0812},
    {"Team": "New York Giants", "Stadium": "MetLife Stadium", "Lat": 40.8135, "Lng": -74.0745},
    {"Team": "New York Jets", "Stadium": "MetLife Stadium", "Lat": 40.8135, "Lng": -74.0745},
    {"Team": "Philadelphia Eagles", "Stadium": "Lincoln Financial Field", "Lat": 39.9008, "Lng": -75.1675},
    {"Team": "Pittsburgh Steelers", "Stadium": "Acrisure Stadium", "Lat": 40.4468, "Lng": -80.0158},
    {"Team": "San Francisco 49ers", "Stadium": "Levi's Stadium", "Lat": 37.4032, "Lng": -121.9698},
    {"Team": "Seattle Seahawks", "Stadium": "Lumen Field", "Lat": 47.5952, "Lng": -122.3316},
    {"Team": "Tampa Bay Buccaneers", "Stadium": "Raymond James Stadium", "Lat": 27.9759, "Lng": -82.5033},
    {"Team": "Tennessee Titans", "Stadium": "Nissan Stadium", "Lat": 36.1665, "Lng": -86.7713},
    {"Team": "Washington Commanders", "Stadium": "Commanders Field", "Lat": 38.9076, "Lng": -76.8645}
    # Note: Coordinates are approximate center of stadium
]

print(f"Loaded {len(nfl_stadiums)} NFL Stadiums.")

Loaded 32 NFL Stadiums.


In [16]:
# --- HELPER FUNCTIONS ---

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in miles between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 3956 # Radius of earth in miles. Use 6371 for kilometers
    return c * r

def find_closest_ihop(gmaps_client, lat, lng):
    """
    Finds the closest IHOP using Google Places API.
    Filters results to ensure 'IHOP' is in the name.
    """
    try:
        # Search for 'IHOP' ranked by distance
        places_result = gmaps_client.places_nearby(
            location=(lat, lng),
            keyword='IHOP',
            rank_by='distance',
            type='restaurant'
        )
        
        if places_result.get('results'):
            # Iterate through results to find one that is actually an IHOP
            for place in places_result['results']:
                name = place.get('name', '')
                if 'IHOP' in name.upper():
                    address = place.get('vicinity')
                    loc = place['geometry']['location']
                    ihop_lat = loc['lat']
                    ihop_lng = loc['lng']
                    
                    return {
                        'IHOP Name': name,
                        'IHOP Address': address,
                        'IHOP Lat': ihop_lat,
                        'IHOP Lng': ihop_lng
                    }
            
            print(f"  ⚠️ Found results but none contained 'IHOP' in name. Top result: {places_result['results'][0].get('name')}")
            return None
        else:
            return None
    except Exception as e:
        print(f"Error finding IHOP: {e}")
        return None

def get_driving_distance(gmaps_client, origin_lat, origin_lng, dest_lat, dest_lng):
    """
    Calculates driving distance and duration using Google Maps Distance Matrix API.
    """
    try:
        result = gmaps_client.distance_matrix(
            origins=(origin_lat, origin_lng),
            destinations=(dest_lat, dest_lng),
            mode="driving",
            units="imperial"
        )
        
        if result['status'] == 'OK':
            element = result['rows'][0]['elements'][0]
            if element['status'] == 'OK':
                distance_text = element['distance']['text']
                distance_val = element['distance']['value'] # meters
                duration_text = element['duration']['text']
                duration_val = element['duration']['value'] # seconds
                
                # Convert meters to miles
                distance_miles = distance_val * 0.000621371
                
                return {
                    'text_dist': distance_text,
                    'val_dist': distance_miles,
                    'text_dur': duration_text,
                    'val_dur': duration_val
                }
        return None
    except Exception as e:
        print(f"Error calculating driving distance: {e}")
        return None

In [17]:
# --- MAIN ANALYSIS LOOP ---

results = []

if API_KEY != 'YOUR_API_KEY_HERE':
    print("Searching for IHOPs and calculating routes... This may take a moment.")
    for team_data in nfl_stadiums:
        team = team_data['Team']
        stadium = team_data['Stadium']
        lat = team_data['Lat']
        lng = team_data['Lng']
        
        print(f"Processing {team}...")
        
        ihop_data = find_closest_ihop(gmaps, lat, lng)
        
        if ihop_data:
            # 1. Haversine Distance
            h_distance = haversine_distance(lat, lng, ihop_data['IHOP Lat'], ihop_data['IHOP Lng'])
            
            # 2. Driving Distance
            drive_data = get_driving_distance(gmaps, lat, lng, ihop_data['IHOP Lat'], ihop_data['IHOP Lng'])
            
            row = {
                'Team': team,
                'Stadium': stadium,
                'Stadium Lat': lat,
                'Stadium Lng': lng,
                'IHOP Name': ihop_data['IHOP Name'],
                'IHOP Address': ihop_data['IHOP Address'],
                'IHOP Lat': ihop_data['IHOP Lat'],
                'IHOP Lng': ihop_data['IHOP Lng'],
                'Haversine Distance (Miles)': round(h_distance, 2)
            }
            
            if drive_data:
                row['Driving Distance (Miles)'] = round(drive_data['val_dist'], 2)
                row['Driving Time'] = drive_data['text_dur']
                row['Distance Delta'] = round(drive_data['val_dist'] - h_distance, 2)
            else:
                row['Driving Distance (Miles)'] = None
                row['Driving Time'] = None
                row['Distance Delta'] = None

            results.append(row)
        else:
            print(f"  No IHOP found for {team}")

    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Sort by Driving Distance (closest first)
    if 'Driving Distance (Miles)' in df.columns:
        df = df.sort_values(by='Driving Distance (Miles)')
    else:
        df = df.sort_values(by='Haversine Distance (Miles)')
    
    # Display top 5 closest
    print("\nTop 5 Closest IHOPs to NFL Stadiums (by Driving Distance):")
    display(df.head())
    
    # Save to CSV (optional)
    df.to_csv('nfl_ihop_distances.csv', index=False)
    print("\nResults saved to 'nfl_ihop_distances.csv'")
else:
    print("⚠️ Analysis skipped. Please provide API Key.")

Searching for IHOPs and calculating routes... This may take a moment.
Processing Arizona Cardinals...
Processing Atlanta Falcons...
Processing Baltimore Ravens...
Processing Buffalo Bills...
Processing Carolina Panthers...
Processing Chicago Bears...
Processing Cincinnati Bengals...
Processing Cleveland Browns...
Processing Dallas Cowboys...
Processing Denver Broncos...
Processing Detroit Lions...
Processing Green Bay Packers...
Processing Houston Texans...
Processing Indianapolis Colts...
Processing Jacksonville Jaguars...
Processing Kansas City Chiefs...
Processing Las Vegas Raiders...
Processing Los Angeles Chargers...
Processing Los Angeles Rams...
Processing Miami Dolphins...
Processing Minnesota Vikings...
Processing New England Patriots...
Processing New Orleans Saints...
Processing New York Giants...
Processing New York Jets...
Processing Philadelphia Eagles...
Processing Pittsburgh Steelers...
Processing San Francisco 49ers...
Processing Seattle Seahawks...
Processing Tampa Ba

,Team,Stadium,Stadium Lat,Stadium Lng,IHOP Name,IHOP Address,IHOP Lat,IHOP Lng,Haversine Distance (Miles),Driving Distance (Miles),Driving Time,Distance Delta
9,Denver Broncos,Empower Field at Mile High,39.7439,-105.0201,Denny's,"1605 Federal Blvd, Denver",39.742991,-105.025625,0.30,0.71,4 mins,0.41
22,New Orleans Saints,Caesars Superdome,29.9511,-90.0812,IHOP,"833 Canal St, New Orleans",29.954541,-90.070412,0.69,1.44,9 mins,0.75
11,Green Bay Packers,Lambeau Field,44.5013,-88.0622,IHOP,"2415 S Oneida St, Green Bay",44.485571,-88.067134,1.11,1.59,5 mins,0.47
2,Baltimore Ravens,M&T Bank Stadium,39.2780,-76.6227,IHOP,"600 E Pratt St, Baltimore",39.286929,-76.607097,1.04,1.59,8 mins,0.55
27,San Francisco 49ers,Levi's Stadium,37.4032,-121.9698,IHOP,"4200 Great America Pkwy, Santa Clara",37.391754,-121.976966,0.88,1.76,7 mins,0.88



Results saved to 'nfl_ihop_distances.csv'


In [18]:
# --- VISUALIZATION ---

if 'df' in locals() and not df.empty:
    # Create a map centered on the US
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

    for index, row in df.iterrows():
        # Stadium Marker (Blue)
        folium.Marker(
            [row['Stadium Lat'], row['Stadium Lng']],
            popup=f"<b>{row['Team']}</b><br>{row['Stadium']}",
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)
        
        # IHOP Marker (Red)
        # Popup includes driving info if available
        popup_text = f"<b>IHOP</b><br>{row['IHOP Address']}<br>"
        popup_text += f"Haversine: {row['Haversine Distance (Miles)']} mi<br>"
        
        if pd.notnull(row.get('Driving Distance (Miles)')):
             popup_text += f"Driving: {row['Driving Distance (Miles)']} mi<br>"
             popup_text += f"Time: {row['Driving Time']}"
        
        folium.Marker(
            [row['IHOP Lat'], row['IHOP Lng']],
            popup=popup_text,
            icon=folium.Icon(color='red', icon='cutlery')
        ).add_to(m)
        
        # Line connecting them
        folium.PolyLine(
            locations=[[row['Stadium Lat'], row['Stadium Lng']], [row['IHOP Lat'], row['IHOP Lng']]],
            color="gray",
            weight=1.5,
            opacity=0.7
        ).add_to(m)

    # Save map to HTML (Fallback for notebook rendering issues)
    map_filename = 'nfl_ihop_map.html'
    m.save(map_filename)
    print(f"\n🗺️ Map saved to '{map_filename}'. Open this file in your browser to view the map if it doesn't appear below.")

    display(m)
else:
    print("⚠️ No data to map.")